In [21]:
#安装并导入Tensorflow和依赖项
!pip install pyyaml h5py  # Required to save models in HDF5 format

import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.6.0


In [22]:
#获取示例数据集
#为了演示如何保存和加载权重，您将使用 MNIST 数据集。为了加快运行速度，请使用前 1000 个样本
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [23]:
#定义模型
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [24]:
#在训练期间保存模型（以 checkpoints 形式保存）
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
32/32 [==============================] - 1s 11ms/step - loss: 1.1869 - sparse_categorical_accuracy: 0.6470 - val_loss: 0.6760 - val_sparse_categorical_accuracy: 0.8030

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 10ms/step - loss: 0.4284 - sparse_categorical_accuracy: 0.8790 - val_loss: 0.5460 - val_sparse_categorical_accuracy: 0.8210

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 7ms/step - loss: 0.3077 - sparse_categorical_accuracy: 0.9200 - val_loss: 0.4497 - val_sparse_categorical_accuracy: 0.8560

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 7ms/step - loss: 0.2189 - sparse_categorical_accuracy: 0.9410 - val_loss: 0.4760 - val_sparse_categorical_accuracy: 0.8410

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 7ms/step - loss: 0.1622 - sparse_ca

In [25]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'checkpoint', 'cp.ckpt.index']

In [26]:
#重新构建一个未经训练的全新模型并基于测试集对其进行评估。未经训练的模型将以机会水平执行（约 10% 的准确率）
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3345 - sparse_categorical_accuracy: 0.1040
Untrained model, accuracy: 10.40%


In [27]:
#从 checkpoint 加载权重并重新评估
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4242 - sparse_categorical_accuracy: 0.8690
Restored model, accuracy: 86.90%


In [28]:
#手动保存权重
#使用 Model.save_weights 方法手动保存权重。默认情况下，
#tf.keras（尤其是 save_weights）使用扩展名为 .ckpt 的 TensorFlow 检查点格式（保存在扩展名为 .h5 的 HDF5 中，保存和序列化模型指南中会讲到这一点）
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4242 - sparse_categorical_accuracy: 0.8690
Restored model, accuracy: 86.90%


In [29]:
#保存整个模型
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 5ms/step - loss: 1.2064 - sparse_categorical_accuracy: 0.6620
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4472 - sparse_categorical_accuracy: 0.8680
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2904 - sparse_categorical_accuracy: 0.9220
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2156 - sparse_categorical_accuracy: 0.9480
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1560 - sparse_categorical_accuracy: 0.9690
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [30]:
#SavedModel 格式是一个包含 protobuf 二进制文件和 TensorFlow 检查点的目录。检查保存的模型目录
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

my_model
assets	keras_metadata.pb  saved_model.pb  variables


In [31]:
#从保存的模型重新加载一个新的 Keras 模型
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [32]:
#使用与原始模型相同的参数编译恢复的模型。尝试使用加载的模型运行评估和预测
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4313 - sparse_categorical_accuracy: 0.8610
Restored model, accuracy: 86.10%
(1000, 10)


In [33]:
#HDF5 格式
#Keras使用 HDF5 标准提供了一种基本的保存格式
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 5ms/step - loss: 1.1699 - sparse_categorical_accuracy: 0.6670
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4182 - sparse_categorical_accuracy: 0.8920
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2901 - sparse_categorical_accuracy: 0.9240
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2219 - sparse_categorical_accuracy: 0.9470
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1536 - sparse_categorical_accuracy: 0.9630


In [34]:
#从该文件重新创建模型
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [35]:
#检查其准确率（accuracy）
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4305 - sparse_categorical_accuracy: 0.8620
Restored model, accuracy: 86.20%
